### dataloading

In [3]:
!nvidia-smi

Wed Aug  4 01:01:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!cp /content/drive/My\ Drive/Kaggle/kaggle.json /content/kaggle.json
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
import os 
os.makedirs('/content/main/', exist_ok=True)
os.chdir('/content/main/')
os.makedirs('/content/main/images-and-metadata', exist_ok=True)
os.makedirs('/content/main/coco', exist_ok=True)
%cd '/content/main/'
!pwd

/content
/content/main


In [6]:
%%bash
# Check nvcc version
nvcc -V
echo
# Check GCC version
gcc --version
echo
# Check the version of torch and cuda packages
pip list | grep "torch\|cuda"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.


cupy-cuda101                  9.1.0
jaxlib                        0.1.69+cuda110
torch                         1.9.0+cu102
torchsummary                  1.5.1
torchtext                     0.10.0
torchvision                   0.10.0+cu102


In [7]:

# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.7.0+cu110 torchvision==0.8.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==1.3.8 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html

# Install mmdetection
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection

!pip install -e .

# install Pillow 7.0.0 back in order to avoid bug in colab
!pip install Pillow==7.0.0

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████▌        | 834.1 MB 1.3 MB/s eta 0:03:47tcmalloc: large alloc 1147494400 bytes == 0x56122ef3c000 @  0x7fabbda67615 0x5611f5f5702c 0x5611f603717a 0x5611f5f59e4d 0x5611f604bc0d 0x5611f5fce0d8 0x5611f5fc8c35 0x5611f5f5b73a 0x5611f5fcdf40 0x5611f5fc8c35 0x5611f5f5b73a 0x5611f5fca93b 0x5611f604ca56 0x5611f5fc9fb3 0x5611f604ca56 0x5611f5fc9fb3 0x5611f604ca56 0x5611f5fc9fb3 0x5611f5f5bb99 0x5611f5f9ee79 0x5611f5f5a7b2 0x5611f5fcde65 0x5611f5fc8c35 0x5611f5f5b73a 0x5611f5fca93b 0x5611f5fc8c35 0x5611f5f5b73a 0x5611f5fc9b0e 0x5611f5f5b65a 0x5611f5fc9d67 0x5611f5fc8c35
     |█████████████████████████████▊  | 1055.7 MB 1.3 MB/s eta 0:01:03tcmalloc: large alloc 1434370048 bytes == 0x561273592000 @  0x7fabbda67615 0x5611f5f5702c 0x5611f603717a 0x5611f5f59e4d 0x5611f604bc0d 0x5611f5fce0d8 0x5611f5fc8c35 0x5611f5f5b73a 0x5611f5fcdf40 0x5611f5fc8c35 0x5611f5f5b73a 0x5611f5fca93b 0x5611f604ca56 0x5611f5fc9f

In [8]:
%cd '/content/main/'

/content/main


In [9]:
!kaggle datasets download -d sreevishnudamodaran/siim-covid19-512-images-and-metadata

 99% 793M/797M [00:28<00:00, 35.9MB/s]
100% 797M/797M [00:28<00:00, 29.0MB/s]


In [10]:
!kaggle datasets download -d sreevishnudamodaran/siim-covid19-coco-512x512-groupkfold

  0% 0.00/1.04M [00:00<?, ?B/s]
100% 1.04M/1.04M [00:00<00:00, 72.2MB/s]


In [11]:
!unzip -qn /content/main/siim-covid19-512-images-and-metadata.zip -d /content/main/images-and-metadata

In [12]:
!unzip -qn /content/main/siim-covid19-coco-512x512-groupkfold.zip -d /content/main/coco

### code

In [ ]:
#!python /content/main/mmdetection/mmdet/utils/collect_env.py

In [1]:
import sys
sys.path.insert(0, "/content/main/mmdetection")

import os
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check MMDetection installation
from mmdet.apis import set_random_seed

# Imports
import mmdet
from mmdet.apis import set_random_seed
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector

import random
import numpy as np
from pathlib import Path

1.7.0+cu110 True
11.0
GCC 7.3


In [2]:
global_seed = 111

def set_seed(seed=global_seed):
    """Sets the random seeds."""
    set_random_seed(seed, deterministic=False)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [3]:
from mmcv import Config

# cfg = Config.fromfile('/kaggle/working/mmdetection/configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py')
# cfg = Config.fromfile("/kaggle/working/mmdetection/configs/vfnet/vfnet_r50_fpn_mstrain_2x_coco.py")
# cfg = Config.fromfile("/kaggle/working/mmdetection/configs/gfl/gfl_r50_fpn_mstrain_2x_coco.py")
# baseline_cfg_path = "/kaggle/working/mmdetection/configs/cascade_rcnn/cascade_rcnn_r50_fpn_20e_coco.py"
baseline_cfg_path = "/content/main/mmdetection/configs/dcn/faster_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py"
cfg = Config.fromfile(baseline_cfg_path)

In [4]:
# model_name = 'vfnet_r50_fpn'
# model_name = 'cascade_rcnn_r50_fpn'
model_name = 'x101_32x4d'
fold = 3
job = 8

# Folder to store model logs and weight files
job_folder = f'/content/main/{model_name}_fold{fold}'
cfg.work_dir = job_folder
cfg.seed = global_seed

if not os.path.exists(job_folder):
    os.makedirs(job_folder)

print("Job folder:", job_folder)

Job folder: /content/main/x101_32x4d_fold3


In [5]:
# Set the number of classes
#for head in cfg.model.roi_head.bbox_head:
#    head.num_classes = 1
cfg.model.roi_head.bbox_head.num_classes = 1
#cfg.model.bbox_head.num_classes = 1

# cfg.gpu_ids = range(1)
cfg.gpu_ids = [0]

# Setting pretrained model in the init_cfg which is required 
# for transfer learning as per the latest MMdetection update
cfg.model.backbone.init_cfg=dict(type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d')
cfg.model.pop('pretrained', None)

cfg.runner.max_epochs = 8 # Epochs for the runner that runs the workflow 
cfg.total_epochs = 8

# Learning rate of optimizers. The LR is divided by 8 since the config file is originally for 8 GPUs
cfg.optimizer.lr = 0.02/8

## Learning rate scheduler config used to register LrUpdater hook
cfg.lr_config = dict(
    policy='CosineAnnealing', # The policy of scheduler, also support CosineAnnealing, Cyclic, etc. Refer to details of supported LrUpdater from https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/lr_updater.py#L9.
    by_epoch=False,
    warmup='linear', # The warmup policy, also support `exp` and `constant`.
    warmup_iters=500, # The number of iterations for warmup
    warmup_ratio=0.001, # The ratio of the starting learning rate used for warmup
    min_lr=1e-07)

# config to register logger hook
cfg.log_config.interval = 200 # Interval to print the log

# Config to set the checkpoint hook, Refer to https://github.com/open-mmlab/mmcv/blob/master/mmcv/runner/hooks/checkpoint.py for implementation.
cfg.checkpoint_config.interval = 1 # The save interval is 1

In [6]:
cfg.dataset_type = 'CocoDataset' # Dataset type, this will be used to define the dataset
cfg.classes = ("Covid_Abnormality",)

cfg.data.train.img_prefix = '/content/main/images-and-metadata/train' # Prefix of image path
cfg.data.train.classes = cfg.classes
cfg.data.train.ann_file = f'/content/main/coco/train_annotations_fold{fold}.json'
cfg.data.train.type='CocoDataset'

cfg.data.val.img_prefix = '/content/main/images-and-metadata/train' # Prefix of image path
cfg.data.val.classes = cfg.classes
cfg.data.val.ann_file = f'/content/main/coco/val_annotations_fold{fold}.json'
cfg.data.val.type='CocoDataset'

cfg.data.test.img_prefix = '/content/main/images-and-metadata/train' # Prefix of image path
cfg.data.test.classes = cfg.classes
cfg.data.test.ann_file =  f'/content/main/coco/val_annotations_fold{fold}.json'
cfg.data.test.type='CocoDataset'

cfg.data.samples_per_gpu = 4 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 2 # Worker to pre-fetch data for each single GPU

In [7]:
# The config to build the evaluation hook, refer to https://github.com/open-mmlab/mmdetection/blob/master/mmdet/core/evaluation/eval_hooks.py#L7 for more details.
cfg.evaluation.metric = 'bbox' # Metrics used during evaluation

# Set the epoch intervel to perform evaluation
cfg.evaluation.interval = 1

# Set the iou threshold of the mAP calculation during evaluation
cfg.evaluation.iou_thrs = [0.5]

# cfg.evaluation.save_best='bbox_mAP_50'

In [8]:
albu_train_transforms = [
    dict(type='ShiftScaleRotate', shift_limit=0.2,
         scale_limit=0.2, rotate_limit=30, p=0.3),
    dict(type='RandomBrightnessContrast', brightness_limit=0.2,
         contrast_limit=0.2, p=0.5),
    dict(type='IAAAffine', shear=(-10.0, 10.0), p=0.2),
    dict(type="Blur", p=0.5, blur_limit=7),
    dict(type='CLAHE', p=0.4),
    dict(type='Equalize', mode='cv', p=0.4),
    dict(
        type="OneOf",
        transforms=[
            dict(type="OpticalDistortion", p=1.0, distort_limit=1.),
            dict(type="GridDistortion", num_steps=5, distort_limit=1., p=1.0),
        ],
        p=0.5,
    ),
    
#     dict(type='MixUp', p=0.2, lambd=0.5),
#     dict(type='RandomRotate90', p=0.5),
#     dict(type='CLAHE', p=0.5),
#     dict(type='InvertImg', p=0.5),
#     dict(type='Equalize', mode='cv', p=0.4),
#     dict(type='MedianBlur', blur_limit=3, p=0.1)
    ]


cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Albu',
        transforms=albu_train_transforms,
        bbox_params=dict(
        type='BboxParams',
        format='pascal_voc',
        label_fields=['gt_labels'],
        min_visibility=0.0,
        filter_lost_elements=True),
        keymap=dict(img='image', gt_bboxes='bboxes'),
        update_pad_shape=False,
        skip_img_without_anno=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels', 'gt_masks'])
]
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(
                type='Normalize',
                mean=[123.675, 116.28, 103.53],
                std=[58.395, 57.12, 57.375],
                to_rgb=True),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img'])
        ])
]

In [9]:
## 4, 8
# cfg.log_level = 'DEBUG'

In [10]:
cfg_path = f'{job_folder}/{Path(baseline_cfg_path).name}'
print(cfg_path)

# Save config file for inference later
cfg.dump(cfg_path)
print(f'Config:\n{cfg.pretty_text}')

/content/main/x101_32x4d_fold3/faster_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py
Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNeXt',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained', checkpoint='open-mmlab://resnext101_32x4d'),
        groups=32,
        base_width=4,
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, True, True, True)),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            stri

In [11]:
model = build_detector(cfg.model,
                       train_cfg=cfg.get('train_cfg'),
                       test_cfg=cfg.get('test_cfg'))
model.init_weights()

/content/main/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '
2021-08-04 13:35:36,327 - mmcv - INFO - load model from: open-mmlab://resnext101_32x4d
2021-08-04 13:35:36,329 - mmcv - INFO - Use load_from_openmmlab loader
2021-08-04 13:35:36,531 - mmcv - WARNING - The model and loaded state dict do not match exactly

missing keys in source state_dict: layer2.0.conv2.conv_offset.weight, layer2.0.conv2.conv_offset.bias, layer2.1.conv2.conv_offset.weight, layer2.1.conv2.conv_offset.bias, layer2.2.conv2.conv_offset.weight, layer2.2.conv2.conv_offset.bias, layer2.3.conv2.conv_offset.weight, layer2.3.conv2.conv_offset.bias, layer3.0.conv2.conv_offset.weight, layer3.0.conv2.conv_offset.bias, layer3.1.conv2.conv_offset.weight, layer3.1.conv2.conv_offset.bias, layer3.2.conv2.conv_offset.weight, layer3.2.conv2.conv_offset.bias, la

In [12]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [13]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2021-08-04 13:35:40,336 - mmdet - INFO - Start running, host: root@be032da601af, work_dir: /content/main/x101_32x4d_fold3
2021-08-04 13:35:40,337 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) EvalHook                           
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) CosineAnnealingLrUpdaterHook       
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook            

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/content/main/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/content/main/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-08-04 13:39:24,025 - mmdet - INFO - Epoch [1][200/859]	lr: 9.944e-04, eta: 2:04:21, time: 1.118, data_time: 0.021, memory: 7743, loss_rpn_cls: 0.2198, loss_rpn_bbox: 0.0161, loss_cls: 0.1409, acc: 95.3000, loss_bbox: 0.1059, loss: 0.4827
2021-08-04 13:43:05,415 - mmdet - INFO - Epoch [1][400/859]	lr: 1.979e-03, eta: 2:00:01, time: 1.107, data_time: 0.010, memory: 7743, loss_rpn_cls: 0.0367, loss_rpn_bbox: 0.0145, loss_cls: 0.1311, acc: 95.9578, loss_bbox: 0.1466, loss: 0.3289
2021-08-04 13:46:47,027 - mmdet - INFO - Epoch [1][600/859]

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 859/859, 7.2 task/s, elapsed: 119s, ETA:     0s

2021-08-04 13:53:35,456 - mmdet - INFO - Evaluating bbox...


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 859/859, 7.2 task/s, elapsed: 119s, ETA:     0s

2021-08-04 15:59:27,992 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*


2021-08-04 15:59:28,553 - mmdet - INFO - Epoch(val) [8][859]	bbox_mAP: 0.5610, bbox_mAP_50: -1.0000, bbox_mAP_75: -1.0000, bbox_mAP_s: 0.0000, bbox_mAP_m: 0.1810, bbox_mAP_l: 0.6390, bbox_mAP_copypaste: 0.561 -1.000 -1.000 0.000 0.181 0.639


DONE (t=0.40s).
Accumulating evaluation results...
DONE (t=0.10s).
 Average Precision  (AP) @[ IoU=0.50:0.50 | area=   all | maxDets=100 ] = 0.561
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.50 | area= small | maxDets=1000 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.50 | area=medium | maxDets=1000 ] = 0.181
 Average Precision  (AP) @[ IoU=0.50:0.50 | area= large | maxDets=1000 ] = 0.639
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=   all | maxDets=100 ] = 0.845
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=   all | maxDets=300 ] = 0.845
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=   all | maxDets=1000 ] = 0.845
 Average Recall     (AR) @[ IoU=0.50:0.50 | area= small | maxDets=1000 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.50 | area=medium | maxDets=1000 ] = 0.527
 Average Recall     (AR) @[ IoU=0.50:0.50 |

In [14]:
# Get the best epoch number
import json
from collections import defaultdict

log_file = f'{job_folder}/None.log.json'

# Source: mmdetection/tools/analysis_tools/analyze_logs.py 
def load_json_logs(json_logs):
    # load and convert json_logs to log_dict, key is epoch, value is a sub dict
    # keys of sub dict is different metrics, e.g. memory, bbox_mAP
    # value of sub dict is a list of corresponding values of all iterations
    log_dicts = [dict() for _ in json_logs]
    for json_log, log_dict in zip(json_logs, log_dicts):
        with open(json_log, 'r') as log_file:
            for line in log_file:
                log = json.loads(line.strip())
                # skip lines without `epoch` field
                if 'epoch' not in log:
                    continue
                epoch = log.pop('epoch')
                if epoch not in log_dict:
                    log_dict[epoch] = defaultdict(list)
                for k, v in log.items():
                    log_dict[epoch][k].append(v)
    return log_dicts

log_dict = load_json_logs([log_file])
# [(print(inner['bbox_mAP']) for inner in item) for item in log_dict]
# [print(item) for item in log_dict[0]]
best_epoch = np.argmax([item['bbox_mAP'][0] for item in log_dict[0].values()])+1
best_epoch

6

In [15]:
#bbox_mAP_copypaste: 0.526 -1.000 -1.000 0.038 0.207 0.606

In [16]:
#!kaggle datasets create -p /content/models

In [22]:
'''baseline_cfg_path = "/content/main/x101_32x4d_fold0/faster_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py"
cfg = Config.fromfile(baseline_cfg_path)

cfg.classes = ("Covid_Abnormality")
cfg.data.test.img_prefix = ''
cfg.data.test.classes = cfg.classes

# cfg.model.roi_head.bbox_head.num_classes = 1
cfg.model.roi_head.bbox_head.num_classes = 1


# Set seed thus the results are more reproducible
cfg.seed = 211
set_random_seed(211, deterministic=False)
cfg.gpu_ids = [0]

cfg.data.test.pipeline=[
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

cfg.test_pipeline = [
            dict(type='LoadImageFromFile'),
            dict(
                type='MultiScaleFlipAug',
                img_scale=(1333, 800),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip', direction='horizontal'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='DefaultFormatBundle'),
                    dict(type='Collect', keys=['img'])
                ])
        ]

# cfg.data.samples_per_gpu = 4
# cfg.data.workers_per_gpu = 4
cfg.model.test_cfg.rcnn.nms.iou_threshold = 0.5
cfg.model.test_cfg.rcnn.score_thr = 0.001
''';

In [24]:
#cfg.dump("/content/faster_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py")

In [25]:
!kaggle datasets create -p /content/models

Starting upload for file cv.txt
100% 978/978 [00:08<00:00, 111B/s]
Upload successful: cv.txt (978B)
Starting upload for file f1_epoch_4.pth
100% 477M/477M [00:34<00:00, 14.6MB/s]
Upload successful: f1_epoch_4.pth (477MB)
Starting upload for file faster_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py
100% 9.53k/9.53k [00:06<00:00, 1.47kB/s]
Upload successful: faster_rcnn_x101_32x4d_fpn_dconv_c3-c5_1x_coco.py (10KB)
Starting upload for file f0_epoch_7.pth
100% 477M/477M [00:38<00:00, 13.1MB/s]
Upload successful: f0_epoch_7.pth (477MB)
Starting upload for file f2_epoch_6.pth
100% 477M/477M [00:33<00:00, 15.1MB/s]
Upload successful: f2_epoch_6.pth (477MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Starting upload for file f3_epoch_6.pth
100% 477M/477M [00:35<00:00, 14.2MB/s]
Upload successful: f3_epoch_6.pth (477MB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gmhost/siim-mmdet-faster-rcnn


In [21]:
#!rm -r /content/main/job8_x101_32x4d_fold2